In [2]:
import pandas as pd
import datetime
import requests
import json
import csv
# import matplotlib as plt

#-*- coding: utf-8 -*-

# kim key no.1 : c15fcbaab76cc525e9342c5e4dda712f
# kim key no.2 : c3268b0918814ae0a2bc298f7ba893a3
# lee key no.1 : 77a87d41d6e2cbe3131a190d5c48514e 
# lee key no.2 : e53650978fc97b47c7f932136ce2c08a

key1 = 'c3268b0918814ae0a2bc298f7ba893a3'
key2 = 'c3268b0918814ae0a2bc298f7ba893a3'

In [ ]:
key1 = 'e53650978fc97b47c7f932136ce2c08a'

def crawl_movie_boxoffice_data(start_date, end_date, key = key1) :
    result_list = []
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json'
    
    for single_date in pd.date_range(start_date, end_date) :
        for multi, rep in zip(['Y', 'Y', 'N', 'N'], ['K', 'F', 'K', 'F']) :
            payload = {
                'key' : key,
                'targetDt' : single_date.strftime('%Y%m%d'),
                'itemPerPage' : '10',
                'multiMovieYn' : multi,
                'repNationCd' : rep
            }
            
            req = requests.get(url, params = payload)

            for item in req.json()['boxOfficeResult']['dailyBoxOfficeList'] :
                temp_list = []
                header_list = []
                for header, value in item.items() :
                    header_list.append(header)
                    temp_list.append(value)
                    
                temp_list.append(single_date)
                header_list.append('CurrentDate')
                
                temp_list.append(multi)
                header_list.append('multi')
                
                temp_list.append(rep)
                header_list.append('Nation')
                
                result_list.append(temp_list)
                
    return pd.DataFrame(result_list, columns = header_list)

movie_boxoffice_list = crawl_movie_boxoffice_data('20140101', '20141231', key = key1)
movie_boxoffice_list.to_csv('movie_boxoffice_2014.csv', index = False)

In [18]:
all_filenames = []
for i in range(2017, 2019) :
    all_filenames.append('movie_boxoffice_'+str(i)+'.csv')
movie_boxoffice_combined = pd.concat([pd.read_csv(f) for f in all_filenames ])

movie_boxoffice_combined.to_csv('movie_boxoffice.csv', index = False, encoding = 'utf-8')

2017
2018


In [19]:
movie_boxoffice_file_2017 = open('movie_boxoffice_2017.csv', 'r', encoding = 'utf-8')
movie_boxoffice_file_2018 = open('movie_boxoffice_2018.csv', 'r', encoding = 'utf-8')

fa=pd.read_csv(movie_boxoffice_file_2017)
fb=pd.read_csv(movie_boxoffice_file_2018)

print(len(fa))
print(len(fb))
with open('movie_boxoffice.csv', 'r', encoding = 'utf-8') as infile :
    fc=pd.read_csv(infile)
    print(len(fc))

14446
14560
29006


In [ ]:
movie_boxoffice_list = pd.read_csv('movie_boxoffice.csv')

In [ ]:
# movie_rating_list = pd.DataFrame({'movieCd' : movie_boxoffice_f.movieCd.drop_duplicates(), 'rating' : 0})
movie_rating_list = pd.DataFrame({'movieCd' : movie_boxoffice_list.movieCd.drop_duplicates(), 'rating' : 0})

In [ ]:
def crawl_movie_detail_data(lists, key = key2) :
    movie_detail_json = dict()
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'
    
    for movieCd in lists['movieCd'] :
        payload = {
            'key' : key,
            'movieCd' : movieCd
        }
        req = requests.get(url, params = payload)
        
        movie_detail_json[movieCd] = req.json()['movieInfoResult']['movieInfo']
        
    return movie_detail_json

movie_detail = crawl_movie_data(movie_boxoffice_list)

with open('movie_detail.json', 'w', encoding = 'utf-8') as output :
    json.dump(movie_detail, output)